In [1]:
import os
import torch as tc
from torch.utils.data import DataLoader
from data_set_.data_loader_ import ISITDataset_gen,make_df_from_data_dir
import matplotlib.pyplot as plt
import numpy as np
from contextlib import nullcontext
import torch.nn as nn
from torch.nn import functional as F  
import math
from dataclasses import dataclass
import inspect

some utils function

In [2]:
def plot_accu(x,y,title,
    suptitile= "Unimodal Classification",
    xlabel='number of events to detection',
    ylabel='probability of correct classification',
    ):
  fig, ax = plt.subplots(1,1,figsize=(5,5))
  fig.suptitle(suptitile, fontsize=16)
  ax.set_title(title,fontsize=12)
  ax.set_ylabel(ylabel,fontsize =12)
  ax.set_xlabel(xlabel,fontsize =12)
  ax.set_ylim([0, 1])
  plt.yticks(np.arange(0, 1+0.05, 0.05))
  plt.grid(True)
  ax.scatter(x,y)
  plt.tight_layout() 
  plt.show()
  return

Discretizer for gpt

In [3]:
class Discretizer:
  def __init__(self,cont_max,token_num):
    self.cont_max = cont_max
    self.token_num = token_num
    self.scale = (token_num-1)/cont_max  
    self.interval  = cont_max/(token_num-1)
  def cont_2_token(self,cont_values):
    assert cont_values.max()<=self.cont_max
    token_values_raw=cont_values*self.scale  
    token_values = token_values_raw.to(dtype=tc.int64)
    assert token_values.max()<=self.token_num
    return token_values
  def token_2_cont(self,token_values):
    token_cont = token_values.to(tc.float32)
    cont_values_raw = token_cont/self.scale
    cont_values = cont_values_raw +\
         tc.rand(token_values.shape)*self.interval
    assert cont_values.max()<=self.cont_max
    return cont_values


#### GPT configuration
get the vocabulary size and setup gpt configuration 

In [4]:

@dataclass
class GPTConfig:
  n_layer:int = 6;  n_head:int = 6;  n_embd:int = 384
  block_size:int = 256 
  bias:bool = False 
  vocab_size:int = None   
  dropout:float = 0.2 # for pretraining 0 is good, for finetuning try 0.1+

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
gpt_conf = GPTConfig()
train_data_ratio = 0.8
df_dict_all,df_dict_train,df_dict_eval = make_df_from_data_dir(train_data_ratio = train_data_ratio)
max_yyy = 0;max_xxx =0;max_ttt=0
print(max_xxx,max_yyy)
for k in df_dict_all.keys():
  x_max = df_dict_all[k]['x'].to_numpy().max()
  y_max = df_dict_all[k]['y'].to_numpy().max()
  t_max = df_dict_all[k]['time_diff'].to_numpy().max()
  print('x',x_max,k)
  print('y',y_max,k)
  if x_max > max_xxx: max_xxx = x_max
  if y_max > max_yyy: max_yyy = y_max
  if t_max > max_ttt: max_ttt = t_max
vocab_size = max(max_xxx,max_yyy)+1  
gpt_conf.vocab_size = vocab_size
# np.sort(df_dict_all['survey_desktop']['time_diff'].to_numpy()
disc = Discretizer(max_ttt,vocab_size)

./data/isit-2024-bits-and-bots/ISIT_Dataset/hlisa_traces
./data/isit-2024-bits-and-bots/ISIT_Dataset/gremlins
./data/isit-2024-bits-and-bots/ISIT_Dataset/za_proxy
./data/isit-2024-bits-and-bots/ISIT_Dataset/survey_desktop
./data/isit-2024-bits-and-bots/ISIT_Dataset/random_mouse_with_sleep_bot
hlisa_traces
gremlins
za_proxy
survey_desktop
random_mouse_with_sleep_bot
hlisa_traces Index(['userId', 'x', 'y', 'eventName', 'userType', 'time_diff'], dtype='object') 398788
unique user 1166 / 398788
gremlins Index(['userId', 'x', 'y', 'eventName', 'userType', 'time_diff'], dtype='object') 185657
unique user 577 / 185657
za_proxy Index(['userId', 'x', 'y', 'eventName', 'userType', 'time_diff'], dtype='object') 46426
unique user 853 / 46426
survey_desktop Index(['userId', 'x', 'y', 'eventName', 'userType', 'time_diff'], dtype='object') 650641
unique user 64743 / 650641
random_mouse_with_sleep_bot Index(['userId', 'x', 'y', 'eventName', 'userType', 'time_diff'], dtype='object') 183365
unique user 

### configuration

In [5]:
# Use Training set for ISIT2024
max_sample_len_train = 70
min_sample_len_train = 3
training_len  =600#* 1000  
eval_len = 10
bs = 128
load_from_checkpoint = True

ckpt_path = 'gpt.ckpt'
if not os.path.isfile(ckpt_path):
  print("cant file ckpt path, train from scratch")
  load_from_checkpoint = False 

cant file ckpt path, train from scratch


Prepare datasets and dataloaders

In [6]:
train_dataset = ISITDataset_gen(df_dict_train,training_len=training_len*bs,
                            min_sample_len=min_sample_len_train,
                            max_sample_len=max_sample_len_train)   
eval_dataset  = ISITDataset_gen(df_dict_eval,training_len=eval_len*bs,
                            min_sample_len=max_sample_len_train,
                            max_sample_len=max_sample_len_train)
train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=bs, shuffle=True)

### model GPT  
some utils function for gpt

In [7]:
def init_weights(module):
  if isinstance(module, nn.Linear):
    tc.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    if module.bias is not None:
        tc.nn.init.zeros_(module.bias)
  elif isinstance(module, nn.Embedding):
    tc.nn.init.normal_(module.weight, mean=0.0, std=0.02)  
def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
  param_dict = {pn: p for pn, p in self.named_parameters()}# start with all candidate params
  param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}# filter out those that do not require grad
  #create optim groups. Any params that is 2D will be weight decayed, otherwise no.
  # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
  decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
  nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
  optim_groups = [
     {'params': decay_params, 'weight_decay': weight_decay},
     {'params': nodecay_params, 'weight_decay': 0.0}
  ]
  num_decay_params = sum(p.numel() for p in decay_params)
  num_nodecay_params = sum(p.numel() for p in nodecay_params)
  print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
  print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
  # Create AdamW optimizer and use the fused version if it is available
  fused_available = 'fused' in inspect.signature(tc.optim.AdamW).parameters
  use_fused = fused_available and device_type == 'cuda'
  extra_args = dict(fused=True) if use_fused else dict()
  optimizer = tc.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
  print(f"using fused AdamW: {use_fused}")
  return optimizer

In [8]:

class LayerNorm(nn.Module):
  """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """
  def __init__(self, ndim, bias):
    super().__init__()
    self.weight = nn.Parameter(tc.ones(ndim))
    self.bias = nn.Parameter(tc.zeros(ndim)) if bias else None
  def forward(self, input):
    return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)
class CausalSelfAttention(nn.Module):
  def __init__(self, c:GPTConfig):
    super().__init__()
    assert c.n_embd % c.n_head == 0
    self.c_attn = nn.Linear(c.n_embd, 3*c.n_embd,bias=c.bias)# qkv projections for all heads,in a batch
    self.c_proj = nn.Linear(c.n_embd, c.n_embd, bias=c.bias)# output projection
    # regularization
    self.attn_dropout=nn.Dropout(c.dropout);self.resid_dropout=nn.Dropout(c.dropout)
    self.dropout = c.dropout
    self.n_head = c.n_head;  self.n_embd = c.n_embd
    # causal mask to ensure that attn is only applied to left in input seq
    self.register_buffer("bias", tc.tril(tc.ones(c.block_size, c.block_size))
                                .view(1, 1, c.block_size, c.block_size))
  def forward(self, x,visible_end):
    B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
    # calculate query, key, values for all heads in batch and move head forward to be the batch dim
    q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
    k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
    att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
    att_raw = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
    
    att_raw[tc.arange(B),:,:,visible_end[0]] = att[tc.arange(B),:,:,visible_end[0]]
    att_raw[tc.arange(B),:,:,visible_end[1]] = att[tc.arange(B),:,:,visible_end[1]]
    att_raw[tc.arange(B),:,:,visible_end[2]] = att[tc.arange(B),:,:,visible_end[2]]
    att = att_raw    #att (128,n_head=6,209,209)   vis_end.max(208)
    #todo
    # breakpoint()
    att = F.softmax(att, dim=-1)
    att = self.attn_dropout(att)
    y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    # breakpoint()
    y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
    # output projection
    y = self.resid_dropout(self.c_proj(y))
    return y
class MLP(nn.Module):
  def __init__(self, c:GPTConfig):
    #c(n_layer=6,n_head=6,n_embd=384,block_size=256,bias=F,vocab_size=65,dropout=0.2)
    super().__init__()
    self.c_fc    = nn.Linear(c.n_embd, 4*c.n_embd, bias=c.bias)
    self.gelu    = nn.GELU()
    self.c_proj  = nn.Linear(4 * c.n_embd, c.n_embd, bias=c.bias)
    self.dropout = nn.Dropout(c.dropout)
  def forward(self, x):
    x = self.c_fc(x)
    x = self.gelu(x)
    x = self.c_proj(x)
    x = self.dropout(x)
    return x
class Block(nn.Module):
  def __init__(self, c:GPTConfig):
    super().__init__()
    self.ln_1 = LayerNorm(c.n_embd, bias=c.bias)
    self.attn = CausalSelfAttention(c)
    self.ln_2 = LayerNorm(c.n_embd, bias=c.bias)
    self.mlp = MLP(c)
  def forward(self, x,visible_end):
    x = x + self.attn(self.ln_1(x),visible_end)
    x = x + self.mlp(self.ln_2(x))
    return x

In [9]:
class GPT(nn.Module):
  def __init__(self, c:GPTConfig):
    super().__init__()
    assert c.vocab_size is not None; assert c.block_size is not None
    self.c = c
    self.wte = nn.Embedding(c.vocab_size, c.n_embd)
    self.lm_head = nn.Linear(c.n_embd, c.vocab_size, bias=False)
    self.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying
    self.wpe = nn.Embedding(c.block_size, c.n_embd)
    self.drop = nn.Dropout(c.dropout)
    self.h = nn.ModuleList([Block(c) for _ in range(c.n_layer)])
    self.ln_f = LayerNorm(c.n_embd, bias=c.bias)
    
    self.apply(init_weights)# init all weights
    for pn, p in self.named_parameters():# apply special scaled init to the residual projections, per GPT-2 paper
      if pn.endswith('c_proj.weight'):
        tc.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * c.n_layer))
  def forward(self, idx, targets=None,terminate_index=None):
    device = idx.device
    b, t = idx.size() #*idx [bs,sentence_len=block_len]#*ex if prompt = "hello",then block len = 5
    assert t <= self.c.block_size, f"cant forward seq of len {t}, block size is only {self.c.block_size}"
    
    if terminate_index != None:
      term_idx = (terminate_index*3)
      term_idx = term_idx.reshape(1,-1)
      visible_end = tc.cat((term_idx-3,term_idx-2,term_idx-1))    
      max_end = visible_end.max()
      assert max_end <= idx.shape[1]
      if max_end == idx.shape[1]:
        visible_end = tc.where(visible_end==max_end,max_end-1,visible_end)
    pos = tc.arange(0, t, dtype=tc.long, device=device) # shape (t)  #*pos =[0]
    # pos = [1,2,3,...,sentence_len] for pos embed
    tok_emb = self.wte(idx) # token embed (b,t)->(b, t, n_embd)
    pos_emb = self.wpe(pos) # pos embed     (t)->   (t, n_embd)
    x = self.drop(tok_emb + pos_emb) # x(b,t,n_embd)
    for block in self.h:
      x = block(x,visible_end)
    x = self.ln_f(x)  
    logits = self.lm_head(x)#(b,t,n_embd=384)->(b,t,vocab_size=65)
    if targets is not None:  # if we are given some desired targets also calculate the loss
      loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.reshape(-1), ignore_index=-1)
    else:
      loss = None
    return logits, loss

#### start training 

In [10]:
idx = 0
model = GPT(gpt_conf).to(device)
#** gpt blocksize  
blocksize = max_sample_len_train*3-1
beta1 = 0.9;  beta2 = 0.99  
learning_rate:float = 1e-3;weight_decay:float = 1e-1;grad_clip:float = 1.0
# optim = tc.optim.Adam(model.parameters(),lr=learning_rate)
optim=configure_optimizers(model,weight_decay, learning_rate, (beta1,beta2), device)
if load_from_checkpoint == False:
  for batch in train_dataloader:
    # for task A. Defense Task,  only the time_diff and position [x,y] elements recorded for each event can be used as input to classifier  
    _, time_diff, pos_x, pos_y, _,terminate_idx = batch  
    #* pos_x,pos_y(bs=128,n_of_events=70),terminate_idx(bs=128),userType(bs=128)
    time_diff_token = disc.cont_2_token(time_diff).to(dtype=tc.float32)  #todo use log for small numbers
    pos_xyt = tc.stack((pos_x,pos_y,time_diff_token),dim=-1).reshape(bs,-1)  #(bs=128,n_of_events*2 = 140)
    if tc.any(time_diff_token>0):
      print(time_diff_token[time_diff_token>0])  #todo  use log at discretizer
    assert  (pos_xyt.max()<=vocab_size) and (pos_xyt.min()>=0) and (time_diff.max()<= max_ttt)
    pos_xyt = pos_xyt.to(device=device,dtype=tc.int64)  #* (bs=128,len=140)
    xx = pos_xyt[:,:blocksize]
    yy = pos_xyt[:,1:blocksize+1]
    assert xx.shape[0]==bs and xx.shape[1]==blocksize and yy.shape[0]==bs and yy.shape[1]==blocksize
    assert xx.max()<vocab_size and yy.max()<vocab_size
    _,loss = model(xx,yy,terminate_idx)
    optim.zero_grad() 
    loss.backward()
    optim.step()
    idx+=1
    # breakpoint()
    print(f"{idx}/{training_len}, term_idx {terminate_idx[0].item()}, {loss.item()=}")
    assert min_sample_len_train<=terminate_idx[0].item()<=max_sample_len_train
  tc.save({'state_dict':model.state_dict()},ckpt_path)
else:
  checkpoint = tc.load(ckpt_path)
  model.load_state_dict(checkpoint['state_dict'])

num decayed parameter tensors: 26, with 12,415,488 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters
using fused AdamW: False
1/600, term_idx 17, loss.item()=7.9279937744140625
2/600, term_idx 69, loss.item()=5.237119674682617
tensor([15.])
3/600, term_idx 48, loss.item()=4.071180820465088
4/600, term_idx 45, loss.item()=3.7404303550720215
tensor([1.])
5/600, term_idx 57, loss.item()=3.6051461696624756
6/600, term_idx 29, loss.item()=3.5577993392944336
tensor([179.])
7/600, term_idx 9, loss.item()=3.4530069828033447
tensor([53.])
8/600, term_idx 40, loss.item()=3.53372859954834
9/600, term_idx 29, loss.item()=3.504119396209717
10/600, term_idx 44, loss.item()=3.161428928375244
11/600, term_idx 52, loss.item()=3.186582326889038
12/600, term_idx 21, loss.item()=3.1978440284729004
13/600, term_idx 58, loss.item()=3.4967141151428223


try to generate some samples using gpt

In [ ]:
# ---------------------------
num_samples = 3 # number of samples to draw
max_new_tokens = blocksize+1 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if tc.cuda.is_available() and tc.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
ptdtype = {'float32':tc.float32,'bfloat16':tc.bfloat16,'float16':tc.float16}[dtype]
ctx = nullcontext() if device == 'cpu' else tc.amp.autocast(device_type=device, dtype=ptdtype)
# encode the beginning of the prompt
x = 21;y = 153;t = 0
end_idx = 3*50
start_xyt = (tc.tensor([x,y,t], dtype=tc.long, device=device)[None, ...])
ini_xyt = tc.zeros((1,max_new_tokens),dtype=tc.long, device=device)
ini_xyt[0,0] = 21   #x 
ini_xyt[0,1] = 153  #y
ini_xyt[0,2] = 0    #t
ini_xyt[0,end_idx] = 56  #x  
ini_xyt[0,end_idx+1] = 321  #y
ini_xyt[0,end_idx+2] = 0    #t
#*** x.shape  [1,3]=  bs, start_sentence_len
# run generation
@tc.no_grad()
def generate(model_, idx, end_idx_, temperature=1.0, top_k=None):
  """Take condition seq of indices (b,t):tc.Long 
  complete seq max_new_tokens times, 
  feed predictions back into model each time."""
  assert idx.size(1) <= model_.c.block_size
  for ii in range(2,end_idx_-1):
    # if seq context grow too long we must crop it at block_size
    idx_cond = idx 
    logits, _ = model_(idx_cond,None,terminate_index=tc.tensor([(end_idx_+3)//3],dtype=tc.int64)) 
    logits = logits[:,[ii],:]
    logits = logits[:, -1, :] / temperature# pluck logits at final step and scale by desired temperature
    if top_k is not None:# optionally crop the logits to only the top k options
      #* topk=200
      v, _ = tc.topk(logits, min(top_k, logits.size(-1)))
      logits[logits < v[:, [-1]]] = -float('Inf')
    # apply softmax to convert logits to (normalized) probabilities
    probs = F.softmax(logits, dim=-1)
    # sample from the distribution
    idx_next = tc.multinomial(probs, num_samples=1)
    # append sampled index to the running sequence and continue
    # idx = tc.cat((idx, idx_next), dim=1)  #add the newly gen last word to the end of the sentence 
    idx[0,ii+1]= idx_next[0,0]
  return idx
with tc.no_grad():
  with ctx:
    for k in range(num_samples):
      gen_xyt = generate(model,ini_xyt,end_idx, temperature=temperature, top_k=top_k)
      print(gen_xyt)
      bs,bl = gen_xyt.shape 
      gen_xyt = gen_xyt.reshape(bs,-1,3)  
      print('---------------')


### evaluation   
evaluate using the unimodal-model trained in 1.a 

In [ ]:

from nets.nn_net import NeuralNetwork
from data_set_.data_loader_ import ISITDataset
max_sample_len_eval = 70  #*70
bs_eval = 1
eval_len = 100
eval_dataset_mlp  = ISITDataset(df_dict_eval,training_len=eval_len*bs_eval,
                            min_sample_len=max_sample_len_eval,
                            max_sample_len=max_sample_len_eval)
eval_dataset  = ISITDataset_gen(df_dict_eval,training_len=eval_len*bs_eval,
                            min_sample_len=max_sample_len_eval,
                            max_sample_len=max_sample_len_eval)

eval_dataloader = DataLoader(eval_dataset, batch_size=bs_eval, 
                             shuffle=True)

category = len(eval_dataset_mlp.idx_2_name)  #5 #['hlisa_traces', 'gremlins', 'za_proxy', 'survey_desktop', 'random_mouse_with_sleep_bot']
ckpt_path = 'model_a1.ckpt'
model_mlp = NeuralNetwork(max_len=max_sample_len_train,
                      output_size=category).to(device)
if not os.path.isfile(ckpt_path):
  raise Exception('you dont have defensive task ckpt, please run "python main.py" first') 
checkpoint = tc.load(ckpt_path)
model_mlp.load_state_dict(checkpoint['state_dict'])
thres_num = 10
conf_thres_list = np.linspace(0,1,thres_num+1)[1:-1]
#*  [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
tot_num_correct = 0; tot_num_samples = 0
test_i = 0
conf_n_correct_list      = np.zeros(len(conf_thres_list))
conf_tot_n_samples_list  = np.zeros(len(conf_thres_list))
n_to_detect_list       = np.zeros(len(conf_thres_list))
for i, name in enumerate(eval_dataset_mlp.idx_2_name):
  if name == 'survey_desktop':
    human_idx = i
assert eval_dataset_mlp.idx_2_name[human_idx] == 'survey_desktop'
blocksize_mlp = 70
for batch in eval_dataloader:
  _, time_diff, pos_x, pos_y, _,terminate_idx = batch
  time_diff=time_diff.to(device);  
  pos_x=pos_x.to(device);          pos_y=pos_y.to(device);
  end_idx_raw = np.random.randint(10,max_sample_len_eval)
  assert 10<= end_idx_raw<max_sample_len_eval

  assert terminate_idx[0]==max_sample_len_eval
  x2 = int(pos_x[0,0].cpu().item())
  y2 = int(pos_y[0,0].cpu().item())
  t2 = int(time_diff[0,0].cpu().item())  #todo   disc 2 token
  start_xyt = (tc.tensor([x2,y2,t2], dtype=tc.long, device=device)[None, ...])
  
  ini_xyt = tc.zeros((1,max_new_tokens),dtype=tc.long, device=device)
  end_idx = end_idx_raw*3
  ini_xyt[0,0] = pos_x[0,0]   #x 
  ini_xyt[0,1] = pos_y[0,0]   #y
  ini_xyt[0,2] = time_diff[0,0]    #t
  ini_xyt[0,end_idx] = pos_x[0,end_idx_raw] #x  
  ini_xyt[0,end_idx+1] = pos_y[0,end_idx_raw] #y
  ini_xyt[0,end_idx+2] = time_diff[0,end_idx_raw]    #t
  # gen_xyt = model.generate(start_xyt, max_new_tokens, temperature=temperature, top_k=top_k)
  gen_xyt = generate(model,tc.clone(ini_xyt),end_idx, temperature=temperature, top_k=top_k)
  bs,bl = gen_xyt.shape 
  gen_xyt = gen_xyt.reshape(bs,-1,3)   
  pos_x_gen = gen_xyt[:,:,0].to(tc.float32)[:,:blocksize_mlp]
  pos_y_gen = gen_xyt[:,:,1].to(tc.float32)[:,:blocksize_mlp]
  time_diff_gen_token = gen_xyt[:,:,2]
  time_diff_gen = disc.token_2_cont(time_diff_gen_token.cpu())[:,:blocksize_mlp].to(device)  #todo use log  or just use continous prediction
  y_target = tc.tensor([human_idx]).to(device)
  for conf_i,conf_thres in enumerate(conf_thres_list):
    max_conf = 0;
    n_to_detect = 0;   
    pos_x2=tc.zeros_like(pos_x_gen);
    pos_y2=tc.zeros_like(pos_y_gen);time_diff2=tc.zeros_like(time_diff_gen)  
    while n_to_detect<max_sample_len_eval and max_conf<conf_thres:
      #**  request more data if conf is lower then threshold
      pos_x2[0,n_to_detect] = pos_x_gen[0,n_to_detect]
      pos_y2[0,n_to_detect] = pos_y_gen[0,n_to_detect]
      time_diff2[0,n_to_detect] = time_diff_gen[0,n_to_detect] 
      y_hat = model_mlp(time_diff2,pos_x2,pos_y2)
      conf_y_hat = tc.softmax(y_hat,dim=1)
      max_conf = conf_y_hat.max(dim=1)[0]
      n_to_detect += 1   
      # breakpoint()
      # print(ii,"max_conf",max_conf)
    _, predictions = y_hat.max(1)
    n_correct = (predictions == y_target).sum();
    n_sample  = predictions.size(0)  #*1
    n_to_detect_list[conf_i] += n_to_detect
    conf_n_correct_list[conf_i] += n_correct
    conf_tot_n_samples_list[conf_i] += n_sample
    tot_num_correct += n_correct
    tot_num_samples += n_sample
  test_i+=1
  print(f"{test_i}/{eval_len}___")



plot 

In [ ]:
avg_n_to_detect_list = n_to_detect_list / eval_len
assert tot_num_samples == (test_i * bs_eval *len(conf_thres_list)) == eval_len*len(conf_thres_list)
print(f"Accuracy on test set: {tot_num_correct/tot_num_samples*100:.2f}")
accu_list = conf_n_correct_list/conf_tot_n_samples_list
suptitle = 'OffenseTask Unimodal'
title = "Accuracy vs Number of Events to Detection"
save_path = f"./img/b_offenseTask_{training_len=}.png"
plot_accu(avg_n_to_detect_list, accu_list, title,save_path,
          suptitile=suptitle,
          xlabel='Number of Events to Detection',
          ylabel='Probability of Correct Classification')